In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
capitals = pd.read_csv("../assets/capitais_brasil.csv")

In [3]:
network = {"county": "Municipal", 
           "state": "Estadual", 
           "federal": "Federal", 
           "public": "Pública"}
level = ["elementary", "middle"] #, "high"] # Tirei o high pois tem só 2017 e 2019

In [4]:
for l in level:
    dfs = []
    for net, esfera in network.items():
        df = pd.read_csv(f"../outputs/ideb_capitals_{l}_{net}.csv")
        df = df.melt(id_vars=["Código do Município"], 
                     var_name="Ano", value_name="IDEB")
        df["Esfera"] = esfera
        df = df.join(capitals.set_index("Código"), on="Código do Município")
        dfs.append(df)
        
ideb = pd.concat(dfs)

In [5]:
ideb

,Código do Município,Ano,IDEB,Esfera,Capitais,Estados,Siglas dos Estados,Regiões
0,1100205,2005,3.3,Municipal,Porto Velho,Rondônia,RO,Norte
1,1200401,2005,2.8,Municipal,Rio Branco,Acre,AC,Norte
2,1302603,2005,2.6,Municipal,Manaus,Amazonas,AM,Norte
3,1400100,2005,4.1,Municipal,Boa Vista,Roraima,RR,Norte
4,1501402,2005,3.1,Municipal,Belém,Pará,PA,Norte
...,...,...,...,...,...,...,...,...
211,4314902,2019,4.0,Pública,Porto Alegre,Rio Grande do Sul,RS,Sul
212,5002704,2019,4.9,Pública,Campo Grande,Mato Grosso do Sul,MS,Centro-Oeste
213,5103403,2019,4.3,Pública,Cuiabá,Mato Grosso,MT,Centro-Oeste
214,5208707,2019,5.4,Pública,Goiânia,Goiás,GO,Centro-Oeste


In [6]:
ideb["Esfera"].value_counts()

Estadual     216
Pública      216
Municipal    208
Federal      128
Name: Esfera, dtype: int64

In [7]:
ideb.to_csv("../outputs/ideb_elementary_middle_anova.csv")

### Label Encoding

In [8]:
ideb_cat = ideb.copy()
for col in ["Esfera", "Capitais", "Estados", "Regiões"]:
    ideb_cat[col] = ideb_cat[col].astype('category').cat.codes

In [9]:
ideb_cat.head(2)

,Código do Município,Ano,IDEB,Esfera,Capitais,Estados,Siglas dos Estados,Regiões
0,1100205,2005,3.3,2,18,21,RO,2
1,1200401,2005,2.8,2,20,0,AC,2


## Sem fitar

In [12]:
ano = "2015"
stats.f_oneway(*[ideb.query("Esfera == @val and Ano == @ano")['IDEB'].replace("-", -1)
                 for val in ideb["Esfera"].unique()])

F_onewayResult(statistic=33.19138188888281, pvalue=1.2479721231465649e-14)

In [13]:
stats.f_oneway(*[ideb.query("Regiões == @val and Ano == @ano")['IDEB'].replace("-", -1)
                 for val in ideb["Regiões"].unique()])

F_onewayResult(statistic=2.321065365231424, pvalue=0.06270853337968)

In [14]:
stats.f_oneway(*[ideb.query("Capitais == @val and Ano == @ano")['IDEB'].replace("-", -1)
                 for val in ideb["Capitais"].unique()])

F_onewayResult(statistic=0.9489148072070523, pvalue=0.5439037096428515)

## Com fit (TODO)

Preciso ver como usa essas funções.

In [ ]:
model = ols('IDEB ~ C(Regiões)', data=ideb_cat).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
aov_table

## Refs

https://www.reneshbedre.com/blog/anova.html